# Homework 2: Information Retrieval
## Instructions
1. Students will form teams of three people each and submit a single homework for each team in the format - ID1_ID2_ID3.ipynb
2. Groups of four are not allowed and you are more than welcome to form groups of two.
2. **Do not write your names anywhere.**
3. For the code part: 
> **Write your code only in the mentioned sections. Do not change the code of other sections**. Do not use any imports unless we say so.
4. For theoretical questions, if any - write your answer in the markdown cell dedicated to this task, in **English**.


#### Deviation from the aforementioned  instructions will lead to reduced grade
---


## Clarifications
1. The same score for the homework will be given to each member of the team.  
2. The goal of this homework is to test your understanding of the concepts presented in the lectures. \
If a topic was not covered in detail during the lecture, you are asked to study it online on your own. 
Anyhow, we provide here detailed explanations for the code part and if you have problems - ask.
3. Questions can be sent to the forum, you are encouraged to ask questions but do so after you have been thinking about your question. 
4. The length of the empty gaps (where you are supposed to write your code) is a recommendation (the amount of space took us to write the solution) and writing longer code will not harm your grade. We do not expect you to use the programming tricks and hacks we used to make the code shorter.   
Having said that, we do encourage you to write good code and keep that in mind - **extreme** cases may be downgraded.  
We also encourage to use informative variable names - it is easier for us to check and for you to understand. 

For your convenience, , the code has a **DEBUG** mode that you may use in order to debug with toy data.  
It is recommended to solve the code in that mode (with efficiency in mind) and then run the code on all the data.
**Do not forget to file the HW with DEBUG == False**.

`Download the data` from [HERE](https://www.kaggle.com/gyani95/380000-lyrics-from-metrolyrics) and put it in the same directory your script is.

Since it is the first time we provide this homework please notify us if there is a bug/something is unclear, typo's exc..

5. We use Python 3.7 for programming.
6. Make sure you have all the packages and functions used in the import section. Most of it is native to Anaconda Python distribution.

### Have fun !

# Imports

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline
from tqdm import tqdm
from typing import List,Dict
from IPython.display import Image
from IPython.core.display import HTML 
from pathlib import Path

In [4]:
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download("stopwords")
nltk.download("punkt")
from string import punctuation, ascii_lowercase
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\elino\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\elino\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Debug
""" you can change this cell """

In [5]:
DEBUG = False
"""
Recommended to start with a small number to get a feeling for the preprocessing with prints (N_ROWS_FOR_DEBUG = 2)
later increase this number for 5*10**3 in order to see that the code runs at reasonable speed. 
When setting Debug == False, our code implements bow.fit() in 15-20 minutes according to the tqdm progress bar. Your solution is not supposed to be much further than that.
"""
N_ROWS_FOR_DEBUG = 5*10**3

# Config

In [6]:
INPUT_FILE_PATH = Path("lyrics.csv")
BOW_PATH = Path("bow.csv")
N_ROWS = N_ROWS_FOR_DEBUG if DEBUG else None
CHUNCK_SIZE = 5 if DEBUG else 5*10**3
tqdm_n_iterations = N_ROWS//CHUNCK_SIZE +1 if DEBUG else 363*10**3//CHUNCK_SIZE + 1
COLS = [5]

## 1.1 Bag of words model
### Implement the following methods:

* `preprocess_sentence`: 
    * Lower case the word
    * Ignores it if it's in the stopwords list
    * Removes characters which are not in the allowed symbols
    * Stems it and appends it to the output sentence
    * Discards words with length <= 1
    
    
* `update_counts_and_probabilities`: 

    * Update self.unigram count (the amount of time each word is in the text)
    * Update self.bigram count (two consecutive word occurances)
    * Update self.trigram count (three consecutive word occurances)
    * Update inverted index: a dictionary with words as keys and the values is a dictionary - {'DocID' : word_count}   
    
* `compute_word_document_frequency`:

   * For each word count the number of docs it appears in. For example , for the word 'apple' -
$$\sum_{i \in docs} I(apple \in doc_i), I := Indicator function$$


* `update_inverted_index_with_tf_idf_and_compute_document_norm`:

    * Update the inverted index (which currently hold word counts) with tf idf weighing. We will compute tf by dividing with the number of words in each document. 
    * As we want to calculate the document norm, incrementally update the document norm. pay attention that later we apply sqrt to it to finish the process.

#### The result of this code is a bag of words model that already counts for TF-IDF weighing

In [7]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
allowed_symbols = set(l for l in ascii_lowercase)

In [8]:
"""This function get an input sentnce and returns new sentnce according to the preprocess rules that where defined"""
def preprocess_sentence(sentence : str) -> List[str]:
    output_sentence = []
    for word in word_tokenize(sentence):                           
        lower_word=word.lower()            #Lower case the word
        if lower_word not in stop_words:   #Ignores it if it's in the stopwords list
            letters=list(lower_word)
            good_letters=[]
            for l in letters:
                if l in allowed_symbols:
                    good_letters.append(l)  #Removes characters which are not in the allowed symbols
            new_word=''.join(good_letters)  #Reconnect the good letters back into one word
            new_word=stemmer.stem(new_word) #stream the word
            if len(new_word)>1:             #Discards words with length <= 1
                output_sentence.append(new_word)
    return output_sentence


def get_data_chuncks() -> List[str]:                           
    for i ,chunck in enumerate(pd.read_csv(INPUT_FILE_PATH, usecols = COLS, chunksize = CHUNCK_SIZE, nrows = N_ROWS)):
        chunck = chunck.values.tolist()
        yield [chunck[i][0] for i in range(len(chunck))] 

class TfIdf:
    def __init__(self):
        self.unigram_count =  Counter()
        self.bigram_count = Counter()
        self.trigram_count = Counter()
        self.document_term_frequency = Counter()
        self.word_document_frequency = {}
        self.inverted_index = {}
        self.doc_norms = {}
        self.n_docs = -1
        self.sentence_preprocesser = preprocess_sentence
        self.bow_path = BOW_PATH

    """This function get an input sentnce and update the unigram, bigram and trigram counter objects. moreover
    the function update the inverted index dict with the number of occurrence for each word in the doc"""  
    def update_counts_and_probabilities(self, sentence :List[str],document_id:int) -> None:
        sentence_len = len(sentence)
        self.document_term_frequency[document_id] = sentence_len
        for i,word in enumerate(sentence):
            self.unigram_count[word]+=1     #update the unigram Counter object that holds for each word how many time it appers in the corpus
            if i<(sentence_len-1):          #edge condition to check that we are not in the last word
                y=word,sentence[i+1]        #take the word and the adjacent word in the sentence
                self.bigram_count[y]+=1     #update the bigram Counter object that holds for each pair of words how many time it appers in the corpus
            if i<(sentence_len-2):          #edge condition to check that we are not in word befor the last word
                z=word,sentence[i+1],sentence[i+2] #take the word and the two adjacent words in the sentence
                self.trigram_count[z]+=1     #update the trigram Counter object that holds for each trio of words how many time it appers in the corpus
            if word not in self.inverted_index:
                word_dict={}                 #initialize new empty dict that will be the value in the inverted_index dict for the key 'word'
                self.inverted_index[word]=word_dict #update the inverted_index dict with the new value 'word_dict'
            word_dict1={}                    #initialize new empty dict and insert to it the doc id as key and the unigram_count of the word as value
            word_dict1[document_id]=self.unigram_count[word] #insert to the empy dict the doc id as key and the unigram_count of the word as value
            current_dict=self.inverted_index[word] #get the current value from inverted_index dict for the key 'word'. this value is a dict with doc id and counts  
            update=current_dict.update(word_dict1) #add to the current dict 'word_dict1' -the new key- value pair that calculated in that loop iteration
            self.inverted_index[word]=current_dict #replace the old value in the inverted_index dict with the new updated dict (with the current doc and word count) 
             
    def fit(self) -> None:
        for chunck in tqdm(get_data_chuncks(), total = tqdm_n_iterations):         
            for sentence in chunck:                                                 
                self.n_docs += 1
                if not isinstance(sentence, str):
                    continue
                sentence = self.sentence_preprocesser(sentence)
                if sentence:
                    self.update_counts_and_probabilities(sentence,self.n_docs)
        self.save_bow() # bow is 'bag of words'
        self.compute_word_document_frequency()                                      
        self.update_inverted_index_with_tf_idf_and_compute_document_norm()
        
    """This function count for each word the number of docs it appears in"""                                                                                 
    def compute_word_document_frequency(self):
        for word in self.inverted_index.keys():     #for each uniqe word, update the word document frequency as the number of docs that the word appers in
            self.word_document_frequency[word]=len(self.inverted_index[word])
            
    """"This function run over all the words in the inverted index dict and for each word and doc replace the word count
    with the tf-idf weighting between the word and the doc. Moreover, the function calculate the docs norm"""        
    def update_inverted_index_with_tf_idf_and_compute_document_norm(self):
        for word in self.inverted_index.keys():           #for each word in the inverted_index keys
            for doc in self.inverted_index[word].keys():  #run over all the the docs that the word apper in
                len_doc=self.document_term_frequency[doc] #get the doc len
                dict_word=self.inverted_index[word]       #get the dict that holds for the word the docs that it appear in it and how many times 
                term_frequency=(dict_word[doc])/len_doc   #calculated the term frequency as the number of times the word appears in the doc divide to the number of words in the doc 
                w_ij=term_frequency*(np.log10((self.n_docs+1)/self.word_document_frequency[word])) #calcluated tf idf weighting: term frequency double in idf(log 10 of the number of docs, divded in the number of docs that the word appers in )               
                first_dict=self.inverted_index[word]       #get the current value from the inverted index dict-a dict with doc as key and counts as values.
                first_dict[doc]=w_ij                       #change the value for the key 'doc' in the word dict from the word count in the dict to the calculated weight   
                if doc not in self.doc_norms:
                     self.doc_norms[doc]=0                 #initialize value for the first time that creating the doc norm
                self.doc_norms[doc]+=(w_ij**2)             #in order to calculated the norm, we will sum up the square weights and in the end take the sqrt 

        for doc in self.doc_norms.keys():
            self.doc_norms[doc] = np.sqrt(self.doc_norms[doc]) 
            
    def save_bow(self):
        pd.DataFrame([self.inverted_index]).T.to_csv(self.bow_path)
                
tf_idf = TfIdf()
tf_idf.fit()


100%|██████████████████████████████████████████████████████████████████████████████████| 73/73 [18:28<00:00, 15.19s/it]


## 1.11 Bag of words model:

1. What is the computational complexity of this model, as a function of the number of docs in the corpus?
2. How can we make this code better in terms running time (parallelization or other topics you find)? 

## 1.2 DocumentRetriever
Not this retriever &#8595;


![dsafdsafsdafdsf](https://cdn3-www.dogtime.com/assets/uploads/2019/10/golden-cocker-retriever-mixed-dog-breed-pictures-cover-1.jpg)

### Implement the following methods:

`reduce_query_to_counts`: given a list of words returns a counter object with words as keys and counts as values.

`rank`: given a query and relevant documents calculate the similarity (cosine or inner product simialrity) between each document and the query.   
Make sure to transform the query word counts to tf idf as well. 

`sort_and_retrieve_k_best`: returns the top k documents.

In [9]:
class DocumentRetriever:
    def __init__(self, tf_idf):
        self.sentence_preprocesser = preprocess_sentence  
        self.vocab = set(tf_idf.unigram_count.keys())
        self.n_docs = tf_idf.n_docs
        self.inverted_index = tf_idf.inverted_index
        self.word_document_frequency = tf_idf.word_document_frequency
        self.doc_norms = tf_idf.doc_norms

    """"This function calculate the simmilarity between each doc and the query using inner product metric or cosine metric"""    
    def rank(self,query : Dict[str,int],documents: Dict[str,Counter],metric: str ) -> Dict[str, float]:
        result = {} # key: DocID , value : float , simmilarity to query
        query_len = np.sum(np.array(list(query.values())))
        query_norm = 0                           #initialize the query norm
        query_w={}                               #initialize dict that holds the query words and their tf-idf weighting 
        for word in query:
            tf=query[word]/query_len              #term frequency of the word is the number of times that the word appers in the query divided in the number of words in the query
            query_w[word]=tf*(np.log10((self.n_docs+1)/self.word_document_frequency[word])) #calcluated tf idf weighting: term frequency double in idf(log 10 of the number of docs, divded in the number of docs that the word appers in )
            query_norm+=(query_w[word])**2        #in order to calculated the norm, we will sum up the square weights and in the end take the sqrt 
            for doc in documents[word].keys():    #for each doc from the docs that the word appears in
                if doc not in result:             #first time the doc appears in the final dict result
                    result[doc]=0                 #initialize the doc value to zero
                dic_word=documents[word]          #get the current value from the documents dict-a dict with doc id as key and weight between the word and the doc as value.
                result[doc]+=(dic_word[doc]*query_w[word]) #sum up the inner_product calculation to the result dict (word-dict weighting duble the word-query weighting)      
        if metric == 'cosine':                    #while using cosine and not inner_product, in the end need to divide the result in the query norm duble the doc norm
            for key in result: result[key] = result[key]/(np.sqrt(query_norm)*self.doc_norms[key])   #doc_norms is already sqrt fron Q.1.1
        return result
        
    """"This function get dict with the doc id and simmilarity and returns list of doc id's that in the top K docs with higher simmilarity"""
    def sort_and_retrieve_k_best(self, scores: Dict[str, float],k :int): 
        return list((dict(Counter(scores).most_common(k))).keys()) 

    """"This function given a list of words returns a counter object with words as keys and counts as values"""
    def reduce_query_to_counts(self, query : List)->  Counter:
        return Counter(query)
        
        
    def get_top_k_documents(self,query : str, metric: str , k = 5) -> List[str]:
        query = self.sentence_preprocesser(query)
        query = [word for word in query if word in self.vocab] # filter nan 
        query_bow = self.reduce_query_to_counts(query)
        relavant_documents = {word : self.inverted_index.get(word) for word in query}
        ducuments_with_similarity = self.rank(query_bow,relavant_documents, metric)
        return self.sort_and_retrieve_k_best(ducuments_with_similarity,k)
        
dr = DocumentRetriever(tf_idf)

In [10]:
from IPython.display import HTML
query = "Better stop dreaming of the quiet life, 'cause it's the one we'll never know And quit running for that runaway bus 'cause those rosy days are few And stop apologizing for the things you've never done 'Cause time is short and life is cruel but it's up to us to change This town called malice"
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/KT6ZtUbVw1M?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

C:\Users\elino\Anaconda3\lib\site-packages\IPython\core\display.py:694: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [11]:
cosine_top_k = dr.get_top_k_documents(query, 'cosine')
print(cosine_top_k)
inner_product_top_k = dr.get_top_k_documents(query, 'inner_product')
print(inner_product_top_k)

[286904, 150547, 107350, 4673, 219069]
[336918, 271639, 175238, 175220, 312772]


In [12]:
for index, song in enumerate(pd.read_csv(INPUT_FILE_PATH,usecols = [5]).iloc[cosine_top_k]['lyrics']):
    sep = "#"*50
    print(F"{sep}\nsong #{index} \n{song} \n{sep}")

##################################################
song #0 
I am thine inmost self
I gaze out upon thy world of coloured lights from the darkness
The darkness behind thine eyes
I am the most ancient one, the creator of the gods
I am the changing and the changless one
I am the source of all that is
In what thou called dreams, I gather my forces
In what thou called reality, I stage my dreams
I am the true god, the only god which is
My dragon magic is sweet, for I will never grant thee thy wishes...
I drink the life essence of those who exist only to serve my will
Your dreams are my life, the bridge between the
Transcendental metamorphosis take place
As certain as the night follows the day
Your mortal human avoidance of "hurt feelings" will cause you to feel -
The pain of death
Realize that the powers of the dreams are five, dimensions unseen...
Astral reflections have immortalized me once, in a time before time
Join those who've risen, bnU rEb kU cAn A, Hekal Tiamat!
What follows is the 

# 1.3 term statistics:
Use "TF-IDF" object that we created earlier and answer the following questions:

1. How many unique words we have?
2. How many potential word bigrams we have? How many actual word bigrams we have? How do you explain this difference?
3. What is the storage size of the input file "lyrics.csv"? What is the output file (bow.csv) size? how do you explain this difference?  

In [13]:
# 1. 
print ('Number of unique words is: ',len(tf_idf.inverted_index))

"""
When we initilize the inverted index we entered each word ones to the dictionary 
and just update the number of the occurance, Therfore all the words there are unique.
In addition we could do that with the 'bow' object like that:
data=pd.read_csv('bow.csv')
df1=pd.DataFrame(data)
print ('Number of unique words is: ',df1.shape[0])
"""

# 2.
print ('Potenial word bigrams (worst case) is: ',((len(tf_idf.inverted_index))**2))
print ('Actual word bigrams is: ',(len(tf_idf.bigram_count)))

"""
We'll compare the actual word biagrmas, which have only unique pairs combinations to the potential word biagram.
Therfore, we want only the number of the unique pairs combinations. If we have V unique words in all of the corpus,
we can say that the count of the potential word bigrams is V**2- This is the upper barrier.
However, in the given data set, the number of pairs combinations is lower than this upper barrier, 
as there are repetitive word pairs combinations in the corpus.
If we are talking not about the unique pairs cpombinations,so for one document we can get n-1 pairs combinations,
when n is the number of all the words in the documents (not only the unique number of words).
Therefore, for D documents we can get (n-1)*D pairs combinations (not unique).
"""

# 3.
print ('The storage size of bow.csv is: ',BOW_PATH.stat().st_size, ' Bytes')
print ('The storage size of lyrcis.csv is: ',INPUT_FILE_PATH.stat().st_size, ' Bytes')

"""
The lyrcis.csv has the full amount of words in the corpus, that includes words that appears more than once.
However, the bow.csv contain each word, that appears more than once in the corpus, only once (with the number of occurrences).
"""

Number of unique words is:  388240
Potenial word bigrams (worst case) is:  150730297600
Actual word bigrams is:  6307797
The storage size of bow.csv is:  249201756  Bytes
The storage size of lyrcis.csv is:  324991390  Bytes


'\nThe lyrcis.csv has the full amount of words in the corpus, that includes words that appears more than once.\nHowever, the bow.csv contain each word, that appears more than once in the corpus, only once (with the number of occurrences).\n'

## 1.4 NgramSpellingCorrector
Now we will implement a Ngarm (character Ngrams) spelling corrector. That is, we have an out of vocabulary word (w) and we want to retrieve the most similar words (in our vocabulary) to this word.
we will model the similarity of two words by-

$$sim(v,w) := prior \cdot likelihood = p(w) \cdot P(v|w) $$ 
$$P(v|w) := JaccardIndex =  \frac{|X \cap Y|}{|X \cup Y|}$$

Where v is an out of vocabulary word (typo or spelling mistake), w is in a vocabulary word, X is the ngram set of v and Y is the ngram set of w.
For example, if n == 3, the set of ngrams for word "banana" is set("ban","ana","nan","ana") = {"ban","ana","nan"}

In order to do it efficently, we will first construct an index from the possible Ngrams we have seen in our corpus to the words that those Ngrams appear in, in order prevent comparing w to all of the words in our corpus.
Then, we will implement a function that computes this similarity.

* Make sure you compute the JaccardIndex efficently!

In [24]:
def get_bigrams(word):
    for ngram in nltk.ngrams(word, 2):
        yield "".join(list(ngram))
        
def get_trigrams(word):
    for ngram in nltk.ngrams(word, 3):
        yield "".join(list(ngram))

In [25]:
class NgramSpellingCorrector:
    def __init__(self, unigram_counts: Counter, get_n_gram: callable):
        self.unigram_counts = unigram_counts
        self.ngram_index = {}
        self.get_n_grams = get_n_gram
        
    '''This function creats the ngram_index. It goes all over the unique words that we have in the unigram dictionary,
       and turn it to his ngrams. The keys in the dictionary are the ngrams, and the values are lists which contains
       all the word which have this ngram'''    
    def build_index(self) -> None:
        for word in self.unigram_counts.keys():                  # Goes all over the word in the unigram
            split_word=set(self.get_n_grams(word))               # Split the word by the get_n_gram which choosen, and turn it to set, which remove duplicate
            for ngram in split_word:                             # Go all over the ngram we got from the splitting
                if ngram not in self.ngram_index:                # If ngram not yed appended to the dictionary ngram_index
                    self.ngram_index[ngram]=[]                   # Initilize the value of this ngram as empty list
                self.ngram_index[ngram].append(word)             # For each ngram key, append to the value the word which certainly contains this ngrm
        
    '''This function takes the word-big string and split it by the ngram which choosen. For each ngram it goes over
        the words which contains this n gram and split the word for calculating the sim between the word and the big
        string. The function saving this sim and finally return the top K word with the biggest sim'''    
    def get_top_k_words(self,word:str,k=5) -> List[str]:
        dict_prob={}                                             # Initilize dictionary which contains the probabilty-sim for each word
        split_str=set(self.get_n_grams(word))                    # Split the big string by the n_gram which, and turn it to set, which remove duplicate
        sum_of_count=sum(self.unigram_counts.values())           # Save the number for the denominator in p(w)
        for ngram in split_str:                                  # Goes over each ngram in the whole word
            if ngram in self.ngram_index:                        # If ngram not in dictionary, it will continue
                for word_n in self.ngram_index[ngram]:               # Goes over each word that contain this ngram
                    if word_n not in dict_prob:
                        split_word=set(self.get_n_grams(word_n))     # Split this word by the n_gram which, and turn it to set, which remove duplicate
                        intersect=len(split_str&split_word)          # Got the number of ngram that are in both the word and the big string
                        union=len(split_str|split_word)              # Got the number of ngram that are in the word and in the big string
                        p_w=(self.unigram_counts[word_n])/(sum_of_count)       # Calculte the prior probability to get the word from all the words in the unigram
                        sim=p_w*(intersect/union)                    # Calculte the sim for the big string and the word
                        dict_prob[word_n]=sim                        # Append the sim to the dictionary under the word (the key)
        return (list((dict(Counter(dict_prob).most_common(k))).keys()))                     # Return only the top K words, that for them the sim is biggest


class BigramSpellingCorrector(NgramSpellingCorrector):
    def __init__(self, unigram_counts: Counter):
        super().__init__(unigram_counts, get_bigrams)
        
        
class TrigramSpellingCorrector(NgramSpellingCorrector):
    def __init__(self, unigram_counts: Counter):
        super().__init__(unigram_counts, get_trigrams)
        

In [26]:
out_of_vocab_word = 'supercalifragilisticexpialidocious'
bigram_spelling_corrector = BigramSpellingCorrector(tf_idf.unigram_count)
bigram_spelling_corrector.build_index()
bigram_spelling_corrector.get_top_k_words(out_of_vocab_word)

['like', 'caus', 'life', 'still', 'time']

In [27]:
trigram_spelling_corrector = TrigramSpellingCorrector(tf_idf.unigram_count)
trigram_spelling_corrector.build_index()
trigram_spelling_corrector.get_top_k_words(out_of_vocab_word)

['life', 'still', 'call', 'listen', 'hous']

## 1.5 Language model
Calculate the log likelihood of a sentence. Once with a bigram markovian langauge model, and once with a trigram model.
for example - the likelihood of the senetence "spiderman spiderman does whatever a spider can" for the bigram model is: 
$$p(spiderman)\cdot p(spiderman|spiderman) \cdot  (does|spiderman) \cdot (whatever|does) \cdot  (a|whatever) \cdot  (spider|a) \cdot (can|spider)$$

And for the trigram model:
$$p(spiderman,spiderman)\cdot p(does|spiderman,spiderman) \cdot  (whatever|spiderman,does) \cdot (a|does,whatever) \cdot  (spider|whatever,a) \cdot  (can|a, spider)$$

Since we do not want a zero probability sentence use Laplace smoothing, as you have seen in the lecture, or here https://en.wikipedia.org/wiki/Additive_smoothing

In [28]:
# for the probability smoothing
NUMERATOR_SMOOTHING = 1 # alpha in https://en.wikipedia.org/wiki/Additive_smoothing
DENOMINATOR_SMOOTHING = 10**4 # d in https://en.wikipedia.org/wiki/Additive_smoothing

'''This function calculates both prior and conditional probabilities in order to calculate
   the log likelihood of a sentence by running over each word in the sentence. 
   This calculation happens for both bigram and triagram langauge models.'''

def sentence_log_probabilty(unigrams : Counter, bigrams  : Counter,trigrams : Counter, sentence: str):
    bigram_log_likelilhood, trigram_log_likelilhood = 0, 0           #initializing the log likelihood for both bigram and triagram langauge models to zero
    words_in_sentence = sentence.split()                             #spiliting the sentence into a list of words
    n_words = len(words_in_sentence)                                 #a variable that holds the Length of the sentence
    for index, word in  enumerate(words_in_sentence):                #runing over each word in the list next to an index
        '''Each prior/conditional probability is smoothed by Laplace Smoothing'''
        if index==0:                                                 #if its the first word in the list
            count_uni_word=unigrams[word]                            #counting how many times the first word appears in our corpus
                                                                     #adding log of the prior propability of the first word to the bigram log likelihood
            bigram_log_likelilhood+=np.log(((count_uni_word)+NUMERATOR_SMOOTHING)/(sum(unigrams.values())+NUMERATOR_SMOOTHING*DENOMINATOR_SMOOTHING))
                                                                     #counting how many times the first+second combination of words appears in our corpus
            count_bi_word=bigrams[(word,words_in_sentence[1])]       #(spider man)
                                                                     #adding log of the prior propability of the first+second combination of words to the trigram log likelihood
            trigram_log_likelilhood+=np.log(((count_bi_word)+NUMERATOR_SMOOTHING)/(sum(bigrams.values())+NUMERATOR_SMOOTHING*DENOMINATOR_SMOOTHING))
                                                                     #counting how many times the first+second+third combination of words appears in our corpus
            count_tri_word=trigrams[(word,words_in_sentence[1],words_in_sentence[2])]
                                                                     #adding log of the first conditional probability to the bigram log likelihood (count_bi_word divided by count_uni_word)
            bigram_log_likelilhood+=np.log((count_bi_word+NUMERATOR_SMOOTHING)/(count_uni_word+NUMERATOR_SMOOTHING*DENOMINATOR_SMOOTHING))
                                                                     #adding log of the first conditional probability to the trigram log likelihood (count_tri_word divided by count_bi_word)
            trigram_log_likelilhood+=np.log((count_tri_word+NUMERATOR_SMOOTHING)/(count_bi_word+NUMERATOR_SMOOTHING*DENOMINATOR_SMOOTHING))
            continue
        if index in range(1,n_words-1):                               #if its not the first/last word
            count_uni_word=unigrams[word]                             #counting how many times the word appears in our corpus
            count_bi_word=bigrams[(word,words_in_sentence[index+1])]  #counting how many times the combination of the current word and next word appears in our corpus
                                                                      #adding log of a conditional probability to the bigram log likelihood (count_bi_word divided by count_uni_word)
            bigram_log_likelilhood+=np.log((count_bi_word+NUMERATOR_SMOOTHING)/(count_uni_word+NUMERATOR_SMOOTHING*DENOMINATOR_SMOOTHING))
        if index in range(1,n_words-2):                               #if its not the first/last two words
                                                                      #counting how many times the combination of the current word and next two words appears in our corpus
            count_tri_word=trigrams[(word,words_in_sentence[index+1],words_in_sentence[index+2])]
                                                                      #adding log of a conditional probability to the trigram log likelihood (count_tri_word divided by count_bi_word)
            trigram_log_likelilhood+=np.log((count_tri_word+NUMERATOR_SMOOTHING)/(count_bi_word+NUMERATOR_SMOOTHING*DENOMINATOR_SMOOTHING))
        
    print(F"Bigram log likelihood is {bigram_log_likelilhood}")
    print(F"Trigram log likelihood is {trigram_log_likelilhood}")
    
sentence = "spider man spider man does whatever a spider can"
sentence_log_probabilty(tf_idf.unigram_count, tf_idf.bigram_count, tf_idf.trigram_count, sentence)

Bigram log likelihood is -80.95155390617043
Trigram log likelihood is -75.24770210005998


## 1.51 Language model: B
For each model what is the next word prediciton for the sentnence "i am"?

In [29]:
'''After the initial word processing operations we found out that the word 'am' is in the inverted index dictionary,
   so we chose to return the word with the max log likelihood of the sentence "i am word" after running over each word 
   in the corpus, for both bigram and trigram language models, instead of choosing a random word.'''

#because P(i)*P(am|i) is permanent in each calculation of the bigram log likelihood -> we will only calculate the coditional probability P(word|am)
#because P(i am) is permanent in each calculation of the trigram log likelihood -> we will only calculate the coditional probability P(word|i am)

max_bi, max_tri=-2**100,-2**100                    #initializing the max probabilty for both models to a large negative number
for word in tf_idf.unigram_count.keys():           #running over each unique word in our corpus
                                                   #P(word|am) with Laplace Smoothing
    prob_bi=np.log(((tf_idf.bigram_count[('am',word)])+NUMERATOR_SMOOTHING)/(tf_idf.unigram_count['am']+NUMERATOR_SMOOTHING*DENOMINATOR_SMOOTHING))
                                                #P(word|i am) with Laplace Smoothing
    prob_tri=np.log(((tf_idf.trigram_count[('i','am',word)])+NUMERATOR_SMOOTHING)/(tf_idf.bigram_count[('i', 'am')]+NUMERATOR_SMOOTHING*DENOMINATOR_SMOOTHING))
    if prob_bi>max_bi:                             #if the bigram probability is larger than the max bigram probabilty
        max_bi=prob_bi                             #change the max probability to the current bigram probability
        next_word_bi=word                          #save the current word as the predicted word in the bigram model
    if prob_tri>max_tri:                           #if the trigram probability is larger than the max trigram probabilty
        max_tri=prob_tri                           #change the max probability to the current trigram probability
        next_word_tri=word                         #save the current word as the predicted word in the trigram model

print ('next word in the biagram model:',next_word_bi)
print ('next word in the trigram model:',next_word_tri)

next word in the biagram model: ll
next word in the trigram model: oh
